<a href="https://colab.research.google.com/github/ichekhovskikh/recommendation-system/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gensim
import os
import collections
import smart_open
import random
import pandas as pd

#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель с использованием Lee Background Corpus. Этот корпус содержит 303 документов, отобранных из службы почтовой рассылки Австралийской радиовещательной корпорации, которая предоставляет текстовые электронные письма с заголовками и охватывает ряд широких тем.

И мы проверим нашу модель на глаз, используя более короткий Lee Corpus, который содержит 34 документов.

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификаторы похожих статей)

In [0]:
# Set file names for train and test data
train_path = 'https://downloader.disk.yandex.ru/disk/9653de8154a2b5b86667bb139ae1f42b5ed14217d3c1ef48f5d0789ea7476777/5c3c7f0b/nwkrVFZsRuFsdebDhfR-d6cHj5l4BUmfu0ikUrSHEavKvid-wyVBOXxTeCLUoLaQqFmijqgSDNbIGtuWXxwOow%3D%3D?uid=248297293&filename=train.csv&disposition=attachment&hash=&limit=0&content_type=application%2Fvnd.ms-excel&fsize=361178&hid=61b3f556a30337ac5b1a700766a4cb93&media_type=spreadsheet&tknv=v2&etag=085db9ca758472d3c64b0d7dd6a97582'
test_path = 'https://downloader.disk.yandex.ru/disk/96d98c2b081115f0ed8ac98242c3b6ff289151f90a9d9b08662dc10231813ed7/5c3c7f30/nwkrVFZsRuFsdebDhfR-d66ojzqYc-hLyj-he1sNQZ_nBzCerRBtYVqPiL4FxyhSreLgPWOQ_jMUUijCk16Llw%3D%3D?uid=248297293&filename=test.csv&disposition=attachment&hash=&limit=0&content_type=application%2Fvnd.ms-excel&fsize=24826&hid=a9a00223c9b53726fd91c9e31efeae7b&media_type=spreadsheet&tknv=v2&etag=8cc5fcdc99aab707ef1a19e5e2623580'

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.) и возвращаем список слов. Для обучения модели нам нужно будет связать тег/номер с каждым документом учебного корпуса. В нашем случае тег - это похожия статья.

In [0]:
def read_corpus(corpus_path, tokens_only=False):
    dataset = pd.read_csv(corpus_path)
    corpus = dataset.iloc[:, :].values
    already_сontained = []
    for i in range(len(corpus[:,0])): 
      
        if corpus[i,0] not in already_сontained:
            already_сontained.append(corpus[i,0])
        else: continue
          
        if tokens_only:
            yield gensim.utils.simple_preprocess(corpus[i,1])
        else:
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(corpus[i,1]), get_tags(corpus[i,0], corpus))

В датасете может появлятся один и тот же текст, с тем же идентификатором, но отличным тегом. Это связано с тем, что для одного и того же текста, может существовать несколько похожих других текстов.

In [0]:
def get_tags(text_id, corpus):
    tags = []
    for i in range(len(corpus[:,0])):   
        if text_id == corpus[i,0]:
            tags.append(corpus[i,2])
    return tags

Получение массива идентификатов:

In [0]:
def get_ids(corpus_path):
    dataset = pd.read_csv(corpus_path)
    corpus = dataset.iloc[:, :].values
    list_ids = []
    for i in range(len(corpus[:,0])): 
        if corpus[i,0] not in list_ids:
            list_ids.append(corpus[i,0])
    return list_ids           

In [0]:
train_corpus = list(read_corpus(train_path))
train_ids = get_ids(train_path)
test_corpus = list(read_corpus(test_path, tokens_only=True))

Давайте посмотрим на учебный корпус:

In [0]:
print(train_corpus[:2])

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

Корпус тестирования выглядит так:

In [0]:
print(test_corpus[:2])

[['the', 'national', 'executive', 'of', 'the', 'strife', 'torn', 'democrats', 'last', 'night', 'appointed', 'little', 'known', 'west', 'australian', 'senator', 'brian', 'greig', 'as', 'interim', 'leader', 'shock', 'move', 'likely', 'to', 'provoke', 'further', 'conflict', 'between', 'the', 'party', 'senators', 'and', 'its', 'organisation', 'in', 'move', 'to', 'reassert', 'control', 'over', 'the', 'party', 'seven', 'senators', 'the', 'national', 'executive', 'last', 'night', 'rejected', 'aden', 'ridgeway', 'bid', 'to', 'become', 'interim', 'leader', 'in', 'favour', 'of', 'senator', 'greig', 'supporter', 'of', 'deposed', 'leader', 'natasha', 'stott', 'despoja', 'and', 'an', 'outspoken', 'gay', 'rights', 'activist', 'cash', 'strapped', 'financial', 'services', 'group', 'amp', 'has', 'shelved', 'million', 'plan', 'to', 'buy', 'shares', 'back', 'from', 'investors', 'and', 'will', 'raise', 'million', 'in', 'fresh', 'capital', 'after', 'profits', 'crashed', 'in', 'the', 'six', 'months', 'to', 

Обратите внимание, что корпус тестирования представляет собой просто список списков и не содержит никаких тегов.

In [17]:
print(train_ids[:10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


# Обучение модели
## Создание объекта Doc2Vec
Теперь мы создадим модель Doc2Vec с векторным размером 50 слов и перебираем учебный корпус 40 раз. Мы устанавливаем минимальную длину слова равной двум, чтобы отбрасывать слова с очень малым количеством вхождений.

Однако это очень маленький набор данных (303 документа) с короткими текстами (несколько сотен слов). Добавление пропусков может иногда помочь с такими маленькими наборами данных.

По сути, словарь - это словарь всех уникальных слов, извлеченных из учебного корпуса.

In [0]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 7.08 s, sys: 204 ms, total: 7.28 s
Wall time: 4.03 s


## Inferring a Vector
Важно отметить, что теперь вы можете вывести вектор для любого фрагмента текста без необходимости переобучать модель, передав список слов в функцию model.infer_vector. Затем этот вектор можно сравнить с другими векторами по косинусному подобию.

In [0]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

array([-0.13778152,  0.10190844,  0.09070924, -0.1291998 ,  0.02204582,
       -0.04395645,  0.15486427, -0.15084767,  0.2042574 , -0.09058648,
       -0.11150775,  0.1230991 ,  0.07102051, -0.04443945,  0.17604499,
       -0.03153215,  0.04492974,  0.08625893,  0.14770153,  0.01397196,
       -0.01415225,  0.16656375,  0.11814791,  0.3571505 ,  0.089607  ,
        0.08551267,  0.04979853,  0.10949661, -0.06877135, -0.01836316,
       -0.00106802, -0.07862338, -0.01490802,  0.23243338,  0.15919799,
        0.19454263, -0.01167523,  0.28749686, -0.08449314, -0.00404665,
       -0.03478434,  0.12410981, -0.12244734,  0.13566194,  0.02238736,
        0.07469752,  0.16304158, -0.08340151, -0.15565938,  0.15476778],
      dtype=float32)

Обратите внимание, что infer_vector() принимает не строку, а список строковых токенов, которые уже должны были быть разбиты на токены так же, как свойство words исходных объектов обучающего документа.

Также обратите внимание, что, поскольку лежащие в основе алгоритмы обучения представляют собой проблему итеративной аппроксимации, в которой используется внутренняя рандомизация, повторные выводы одного и того же текста будут возвращать слегка разные векторы.

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

In [0]:
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


По сути, более 95% предполагаемых документов оказываются наиболее похожими на себя, и примерно в 5% случаев они по ошибке наиболее похожи на другие документы. проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя с пользой и согласованностью, хотя и не является реальным значением «точности».

Это здорово и не совсем удивительно. Мы можем взглянуть на пример:

In [0]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (302): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

Обратите внимание, что наиболее похожий документ (как правило, тот же текст) имеет показатель сходства, приближающийся к 1,0. Тем не менее, оценка сходства для документов второго ранга должна быть значительно ниже (при условии, что документы на самом деле разные), и рассуждение становится очевидным, когда мы исследуем сам текст.

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [0]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (42): «pope john paul ii urged delegates at major summit on sustainable growth on sunday to pursue development that protects the environment and social justice in comments to tourists and the faithful at his summer residence southeast of rome the pope said god had put humans on earth to be his administrators of the land to cultivate it and take care of it in world ever more interdependent peace justice and the safekeeping of creation cannot but be the fruit of joint commitment of all in pursuing the common good john paul said»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (186, 0.7243496775627136): «united nationals secretary general kofi annan has accepted the nobel peace prize in the norwegian capital oslo declaring that to save one life is to save humanity itself mr annan told gala audience the world must respect the individual whose fundamental rights he says have been sacrificed too often for the good of the state the year old un chi

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
